# Dive Prediction - Hidden Markov Model

*Predicting Seabird Diving Behaviour from GPS data*

This notebook trains a HMM to predict seabirds' dives.

HMM' characteristics:

* *Number of modes* : 3

<div class="alert alert-info">
 ⚠️ Notebook using R
</div>

In [8]:
library(momentuHMM)
library(pracma)
library(lubridate)

In [9]:
options(stringsAsFactors = FALSE)

data_test = read.table("./../data/LB_test.csv", sep = ',', header = TRUE)
data_test$datetime <- as.POSIXct(data_test$datetime, tz = 'GMT')

In [10]:
## 
resolution = 30

In [11]:
## Resample at Resolution
data_test_new <- data.frame()

for (id in unique(data_test$trip)){
    t <- data_test[data_test$trip == id,]
    
    idx = seq(1, nrow(t), by = resolution)
    traj <- t[idx,c("trip", "datetime", "lon", "lat")]
    
    # dives as maximum in window 
    traj$dive = sapply(idx, function(i){
            max(t$dive[i:(i+resolution-1)], na.rm = TRUE)
        })
    
    traj$gaps = sapply(idx, function(i){
        mean((t$gaps[i:(i+resolution-1)] == 'True'), na.rm = TRUE)
    })
    
    
    data_test_new <- rbind(data_test_new, traj)
}
data_test_new

,trip,datetime,lon,lat,dive,gaps
,<chr>,<dttm>,<dbl>,<dbl>,<int>,<dbl>
1,P1109_15_LB_T4,2009-12-03 19:30:28,-77.26298,-11.77297,0,0.5000000
31,P1109_15_LB_T4,2009-12-03 19:30:58,-77.26411,-11.77592,0,0.5000000
61,P1109_15_LB_T4,2009-12-03 19:31:28,-77.26713,-11.77523,0,0.5000000
91,P1109_15_LB_T4,2009-12-03 19:31:58,-77.26590,-11.77000,0,0.5000000
121,P1109_15_LB_T4,2009-12-03 19:32:28,-77.26577,-11.76484,0,0.5000000
151,P1109_15_LB_T4,2009-12-03 19:32:58,-77.26665,-11.75982,0,0.5000000
181,P1109_15_LB_T4,2009-12-03 19:33:28,-77.26899,-11.75599,0,0.5000000
211,P1109_15_LB_T4,2009-12-03 19:33:58,-77.27107,-11.75359,0,0.5000000
241,P1109_15_LB_T4,2009-12-03 19:34:28,-77.27160,-11.74979,1,0.6000000


# Fit HMM

In [12]:
# prep data for HMM
rawData <- data_test_new[,c("trip","lon","lat", "datetime", "dive", "gaps")]
colnames(rawData) <- c("ID", "lon", "lat", "datetime", "dive", "gaps")
birdData <- prepData(data=rawData, type = "LL", coordNames = c("lon", "lat"))
birdData

ID,step,angle,datetime,dive,gaps,x,y
<fct>,<dbl>,<numeric>,<dttm>,<int>,<dbl>,<dbl>,<dbl>
P1109_15_LB_T4,0.348511304,NA,2009-12-03 19:30:28,0,0.5000000,-77.26298,-11.77297
P1109_15_LB_T4,0.337886826,-1.43595076,2009-12-03 19:30:58,0,0.5000000,-77.26411,-11.77592
P1109_15_LB_T4,0.594568622,-1.57377645,2009-12-03 19:31:28,0,0.5000000,-77.26713,-11.77523
P1109_15_LB_T4,0.570193467,0.20418566,2009-12-03 19:31:58,0,0.5000000,-77.26590,-11.77000
P1109_15_LB_T4,0.564041838,0.19555079,2009-12-03 19:32:28,0,0.5000000,-77.26577,-11.76484
P1109_15_LB_T4,0.494447122,0.37013793,2009-12-03 19:32:58,0,0.5000000,-77.26665,-11.75982
P1109_15_LB_T4,0.349634654,0.16657668,2009-12-03 19:33:28,0,0.5000000,-77.26899,-11.75599
P1109_15_LB_T4,0.423478715,-0.57198846,2009-12-03 19:33:58,0,0.5000000,-77.27107,-11.75359
P1109_15_LB_T4,0.287004743,1.02091255,2009-12-03 19:34:28,1,0.6000000,-77.27160,-11.74979


## 3 states with gaps

In [13]:
# parameters initialization
## cluster K-Means for initialization
### STEP
clusterBird_step <- kmeans(na.omit(data.frame(birdData$step)), 3)
muS_1 <- max(clusterBird_step$centers)
muS_2 <- median(clusterBird_step$centers) 
muS_3 <- min(clusterBird_step$centers) 
sdS_1 <- sd(na.omit(birdData$step)[clusterBird_step[[1]] == which(clusterBird_step$centers == max(clusterBird_step$centers))])
sdS_2 <- sd(na.omit(birdData$step)[clusterBird_step[[1]] == which(clusterBird_step$centers == median(clusterBird_step$centers))])
sdS_3 <- sd(na.omit(birdData$step)[clusterBird_step[[1]] == which(clusterBird_step$centers == min(clusterBird_step$centers))])

### ANGLE
## for von mises
kappaA_1 <- 4
kappaA_2 <- 2
kappaA_3 <- 0.1

# ### GAPS
# ## Bernouilli
# p_1 <- 0.8
# p_2 <- 0.2
# p_3 <- 0.2

### ZERO MASS
zeroMass <- length(which(birdData$step == 0))/nrow(birdData) #we need to include zeroMass parameters

### FIT MODEL
stateNames <- c("fly","observe", "dive")
dist = list(step = "gamma", angle = "vm")

anglePar0 <- c(kappaA_1, kappaA_2, kappaA_3) 
stepPar0 <- c(muS_1, muS_2, muS_3, sdS_1, sdS_2, sdS_3, zeroMass, zeroMass, zeroMass)
# gapsPar0 <-  c(p_1, p_3) 

# formula for transition probabilities
formula <- ~ gaps
# formula <- ~ 1

In [14]:
m <- fitHMM(data = birdData, nbStates = 3, dist = dist,
            Par0 = list(step = stepPar0, angle = anglePar0), stateNames = stateNames,
            formula = formula)


Fitting HMM with 3 states and 2 data streams

-----------------------------------------------------------------------


 step ~ gamma(mean=~1, sd=~1, zeromass=~1)

 angle ~ vm(concentration=~1)


 Transition probability matrix formula: ~gaps


 Initial distribution formula: ~1


DONE



In [18]:
data <- m$data
prob = stateProbs(m)
data$prediction = prob[, 'dive']

In [ ]:
write.table(data, file = paste0('`./../results/LB_hmm_30s.csv'), sep = ',', row.names = FALSE)

In [ ]:
TP <- NULL
FP <- NULL
for (p in unique(data$prediction)){
    all_estim <- 1*(data$prediction > p)
    
    true_positive <- mean(all_estim[data$dive == 1])
    true_negative <- 1-mean(all_estim[data$dive == 0])
    
    TP <- c(TP,true_positive)
    FP <- c(FP,1-true_negative)
}

In [ ]:
plot(FP,TP, xlim = c(0,1), ylim = c(0,1))

# Dive Prediction

In [ ]:
for (trip in unique(rawData$ID)){
  path = paste0('./../results/hmm/figures/', trip, '_', resolution,'s_GNP.png')
#   png(path)
  plot(m, animals = trip, ask = FALSE)
#   dev.off()
}

## compute loss

In [ ]:
loss <- function(x,y,weight){
    x[x == 0] <- 1e-5
    x[x == 1] <- 1-1e-5
    return(-mean(weight*y*log(x) + (1-y)*log(1-x)))
}

In [ ]:
loss(data$prediction, data$dive, 30)